### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class

In [7]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [2]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [3]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [4]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [8]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [9]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [10]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [22]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several reasons in favor of choosing AutoGen for your new AI Agent project:

1. **Configurable Agents**: AutoGen allows for the creation of agents with customizable behaviors, enabling you to tailor them to specific project requirements.

2. **Multi-Agent Workflows**: It supports multi-agent conversational workflows, allowing for complex interactions and asynchronous communication between multiple agents, which can enhance the application's functionality.

3. **Human-Machine Collaboration**: The framework facilitates effective collaboration between humans and machines, making it easier to integrate human oversight where necessary.

4. **Built-In Tool and API Support**: AutoGen has built-in support for utilizing tools and integrating APIs, which can streamline the process of connecting your agents with external services and data sources.

5. **Event-Driven Architecture**: Its event-driven, asynchronous architecture allows for scalable and efficient workflows, which can improve responsiveness and resource management in your application.

These features make AutoGen a robust choice for developing sophisticated AI agents with the ability to handle various tasks and interactions efficiently. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen in AI projects:

1. **Poor Documentation**: The documentation for AutoGen can be difficult to read, lacking sufficient examples, which may hinder developers' ability to fully utilize its features.

2. **Limited Functionality**: Certain features, such as structured outputs, may not work as intended, leading to frustrations during development.

3. **Confusion with Similar Tools**: There may be ambiguity in distinguishing AutoGen from similar applications like AG2, which could lead to confusion in choosing the appropriate tool for the project.

4. **Learning Curve**: Due to the lack of clear guides and examples, there may be a steeper learning curve for new users, potentially delaying project timelines.

These factors could lead to increased development time and complexity in the project. 

TERMINATE



## Decision:

Based on the analysis of the pros and cons of AutoGen, I recommend moving forward with using AutoGen for the project. 

The benefits of configurable agents, multi-agent workflows, and human-machine collaboration align well with our project's goals of creating scalable and efficient AI agents. Despite the concerns regarding poor documentation and the learning curve, the advantages in functionality and architecture could outweigh these challenges. The built-in support for tools and APIs, along with the event-driven architecture, suggests that AutoGen has the potential to meet our project's needs effectively.

TERMINATE

In [11]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [12]:
await host.stop()